In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('final.csv')

In [11]:
import cv2

In [12]:
import numpy as np

In [13]:
import pickle

In [14]:
with open('RF_final.pkl', 'rb') as f:
    m = pickle.load(f)

C:\Users\scien\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:463: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.6.1 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
C:\Users\scien\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\sklearn\base.py:463: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.6.1 when using version 1.8.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [15]:
import mediapipe as mp
import time

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

x_coord = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
y_coord = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
c = 5
def print_result(result, output_image, timestamp_ms):
    global x_coord, y_coord
    # set coords when a hand is found, otherwise reset and mark absent
    if result.hand_landmarks and len(result.hand_landmarks) > 0:
        lm_list = result.hand_landmarks[0]
        for i, lm in enumerate(lm_list):
            x_coord[i] = lm.x
            y_coord[i] = lm.y

        present = True
        print(f'coordinates: ({x_coord[0]}, {y_coord[0]})')
    else:
        x_coord = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        y_coord = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
        present = False
    videoFeed(output_image, x_coord, y_coord, present)
old_pred = ''
pred = ''
def videoFeed(img, Xc, Yc, present):
    global c, old_pred, pred
    dic = {}
    n_frame = img.numpy_view()
    new_frame = np.copy(n_frame)
    new_frame = cv2.cvtColor(new_frame, cv2.COLOR_RGB2BGR)
    if present:
        h, w = new_frame.shape[:2]
        x1 = int(min(Xc) * w)
        y1 = int(min(Yc) * h)
        x2 = int(max(Xc) * w)
        y2 = int(max(Yc) * h)
        cv2.rectangle(new_frame, (x1-10, y1-10), (x2+10, y2+10), (255, 255, 255), 2)
        text = f'X: {Xc[0]:.2f}, Y: {Yc[0]:.2f}'
        cv2.putText(new_frame, text, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)
        gray_fil = cv2.cvtColor(new_frame, cv2.COLOR_BGR2GRAY)
    
        cx = int((x1 + x2) / 2)
        cy = int((y1 + y2) / 2)
        l = max(x2 - x1, y2 - y1) + 20
        x_start = max(cx - l//2, 0)
        y_start = max(cy - l//2, 0)
        x_end = min(cx + l//2, w)
        y_end = min(cy + l//2, h)
        cropped = gray_fil[y_start:y_end, x_start:x_end]
        resized = cv2.resize(cropped, (28, 28))
        bin_fil = cv2.threshold(resized, 160, 255, cv2.THRESH_BINARY)[1]

        for i in range(28):
            for j in range(28):
                dic[f'{i,j}'] = bin_fil[i,j]/255.0
        df = pd.DataFrame(dic,index=[0])
        
        if c == 10:
            pred = m.predict(df)[0]
            old_pred = pred
            #print(pred)
            c = 0
        elif present:
            c += 1
        
        pred_show = f'Prediction: {old_pred}'
        cv2.putText(new_frame, pred_show, (x2+10, y2+30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1)

    cv2.imshow('Live Video Feed', new_frame)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        #cv2.destroyAllWindows()
        
cap = cv2.VideoCapture(0)
options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.LIVE_STREAM, min_hand_detection_confidence=0.5, min_tracking_confidence=0.5, num_hands=1,
    result_callback=print_result)
with HandLandmarker.create_from_options(options) as landmarker:
    
    last_timestamp_ms = 0
    

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        now_ms = int(time.monotonic() * 1000)
        if now_ms <= last_timestamp_ms:
            now_ms = last_timestamp_ms + 1
        last_timestamp_ms = now_ms
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=gray)
        landmarker.detect_async(mp_image, last_timestamp_ms)




coordinates: (0.483467161655426, 0.9189825057983398)
coordinates: (0.49323320388793945, 0.8806682825088501)
coordinates: (0.4895722270011902, 0.8815768361091614)
coordinates: (0.4876909554004669, 0.8674732446670532)
coordinates: (0.48394328355789185, 0.8591695427894592)
coordinates: (0.4836532771587372, 0.8602234721183777)
coordinates: (0.48009148240089417, 0.8530876636505127)
coordinates: (0.4795784056186676, 0.8491498827934265)
coordinates: (0.47924068570137024, 0.8484247326850891)
coordinates: (0.4783591628074646, 0.8474057912826538)
coordinates: (0.4780235290527344, 0.8449985980987549)
coordinates: (0.47767433524131775, 0.8439447283744812)
coordinates: (0.47724440693855286, 0.843324601650238)
coordinates: (0.4777229130268097, 0.8446464538574219)
coordinates: (0.477423757314682, 0.8439040184020996)
coordinates: (0.4780674874782562, 0.8458719253540039)
coordinates: (0.4783463478088379, 0.8458119034767151)
coordinates: (0.47838276624679565, 0.8458881378173828)
coordinates: (0.48004025

KeyboardInterrupt: 

In [16]:
cap.release()

import mediapipe as mp
import time

BaseOptions = mp.tasks.BaseOptions
HandLandmarker = mp.tasks.vision.HandLandmarker
HandLandmarkerOptions = mp.tasks.vision.HandLandmarkerOptions
HandLandmarkerResult = mp.tasks.vision.HandLandmarkerResult
VisionRunningMode = mp.tasks.vision.RunningMode

x_coord = 0
y_coord = 0
# Create a hand landmarker instance with the live stream mode:
def print_result(result, output_image, timestamp_ms):
    if result.hand_landmarks:
        for hand_idx, lm_list in enumerate(result.hand_landmarks):
            for i, lm in enumerate(lm_list):
                if i == 8:
                    x_coord = lm.x
                    y_coord = lm.y
                    print(f'Index Finger Tip coordinates: ({x_coord}, {y_coord})')
                    if x_coord == 0 and y_coord == 0:
                        videoFeed(output_image, x_coord, y_coord, present=False)
                    else:
                        videoFeed(output_image, x_coord, y_coord, present=True)

def videoFeed(img, Xc, Yc, present):
    frame = img.numpy_view()
    new_frame = np.copy(frame)
    y_, x_ = new_frame.shape[:2]
    x = int(Xc * x_)
    y = int(Yc * y_)
    if present:
        color = (0, 255, 0)  # Green for presence
        radius = 5
        cv2.circle(new_frame, (x, y), radius, color, -1)
        cv2.imshow('Live Video Feed', new_frame)
    else:
        

    if cv2.waitKey(1) & 0xFF == ord('q'):
        cap.release()
        cv2.destroyAllWindows()
        quit()
        

options = HandLandmarkerOptions(
    base_options=BaseOptions(model_asset_path='hand_landmarker.task'),
    running_mode=VisionRunningMode.LIVE_STREAM, min_hand_detection_confidence=0.5, min_tracking_confidence=0.5, num_hands=1,
    result_callback=print_result)
with HandLandmarker.create_from_options(options) as landmarker:
  # The landmarker is initialized. Use it here.
  # ...
    cap = cv2.VideoCapture(0)
    last_timestamp_ms = 0
    # Check if the video source was opened successfully
    if not cap.isOpened():
        print("Error: Could not open video source.")
        exit()

    while True:
        # Read a frame from the video source
        ret, frame = cap.read()

        # Check if the frame was retrieved successfully
        if not ret:
            print("Can't receive frame (stream end?). Exiting ...")
            break
        #cv2.imshow('Live Video Feed', frame)

        #if cv2.waitKey(1) & 0xFF == ord('q'):
        #    break
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        now_ms = int(time.monotonic() * 1000)
        if now_ms <= last_timestamp_ms:
            now_ms = last_timestamp_ms + 1
        last_timestamp_ms = now_ms
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=gray)
        landmarker.detect_async(mp_image, last_timestamp_ms)


